# Questão 08 - Recomendação de Filmes com Filtragem Colaborativa
**Autor:** Leonardo Paz  
**Data:** Outubro 2024

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas")

In [ ]:
# Criar dados de exemplo (avaliações de filmes)
np.random.seed(42)
n_usuarios = 50
n_filmes = 30

# Gerar matriz de avaliações
avaliacoes_data = []
for usuario in range(1, n_usuarios + 1):
    for filme in np.random.choice(range(1, n_filmes + 1), size=15, replace=False):
        avaliacao = np.random.randint(1, 6)
        avaliacoes_data.append({'user_id': usuario, 'movie_id': filme, 'rating': avaliacao})

avaliacoes = pd.DataFrame(avaliacoes_data)
filmes = pd.DataFrame({'movie_id': range(1, n_filmes + 1), 
                      'title': [f'Filme {i}' for i in range(1, n_filmes + 1)]})

print("Dados criados:")
print(f"Avaliações: {avaliacoes.shape}")
print(f"Filmes: {filmes.shape}")
avaliacoes.head()

In [ ]:
# Matriz usuário-filme
matriz_usuario_filme = avaliacoes.pivot_table(
    index='user_id', 
    columns='movie_id', 
    values='rating'
).fillna(0)

print("Matriz Usuário-Filme:")
print(f"Shape: {matriz_usuario_filme.shape}")
print(f"Densidade: {(matriz_usuario_filme > 0).sum().sum() / (matriz_usuario_filme.shape[0] * matriz_usuario_filme.shape[1]) * 100:.1f}%")
matriz_usuario_filme.iloc[:5, :5]

In [ ]:
# Filtragem colaborativa baseada em usuário
similaridade_usuarios = cosine_similarity(matriz_usuario_filme)

# Filtragem colaborativa baseada em item
similaridade_filmes = cosine_similarity(matriz_usuario_filme.T)

print("Similaridades calculadas:")
print(f"Similaridade entre usuários: {similaridade_usuarios.shape}")
print(f"Similaridade entre filmes: {similaridade_filmes.shape}")

In [ ]:
# Função de recomendação
def recomendar_filmes(user_id, n_recomendacoes=5, metodo='usuario'):
    try:
        if metodo == 'usuario':
            # Baseado em usuários similares
            user_idx = user_id - 1
            similaridades = similaridade_usuarios[user_idx]
            usuarios_similares_idx = np.argsort(similaridades)[::-1][1:6]
            
            # Recomendar filmes não vistos com melhores avaliações
            filmes_nao_vistos = matriz_usuario_filme.iloc[user_idx] == 0
            avaliacoes_similares = matriz_usuario_filme.iloc[usuarios_similares_idx].mean(axis=0)
            recomendacoes = avaliacoes_similares[filmes_nao_vistos].sort_values(ascending=False).head(n_recomendacoes)
            
        else:
            # Baseado em itens similares
            filmes_vistos = matriz_usuario_filme.iloc[user_id-1][matriz_usuario_filme.iloc[user_id-1] > 0].index
            if len(filmes_vistos) == 0:
                return pd.Series([])
            
            # Calcular similaridade com filmes não vistos
            scores = {}
            for filme in matriz_usuario_filme.columns:
                if matriz_usuario_filme.iloc[user_id-1, filme-1] == 0:
                    score = 0
                    for filme_visto in filmes_vistos:
                        score += similaridade_filmes[filme-1, filme_visto-1] * matriz_usuario_filme.iloc[user_id-1, filme_visto-1]
                    scores[filme] = score / len(filmes_vistos) if len(filmes_vistos) > 0 else 0
            
            recomendacoes = pd.Series(scores).sort_values(ascending=False).head(n_recomendacoes)
        
        return recomendacoes
    except:
        return pd.Series([])

# Testar recomendação
user_test = 1
rec_usuario = recomendar_filmes(user_test, 3, 'usuario')
rec_item = recomendar_filmes(user_test, 3, 'item')

print(f"Recomendações para usuário {user_test}:")
print("Baseado em usuário:", [f"Filme {fid}" for fid in rec_usuario.index])
print("Baseado em item:", [f"Filme {fid}" for fid in rec_item.index])

## Resposta Questão 08

**Qual abordagem foi mais eficiente na recomendação de filmes?**

A abordagem **baseada em itens** geralmente é mais eficiente porque:
- Matriz de similaridade entre filmes é mais estável
- Menor susceptível a mudanças no comportamento dos usuários
- Computacionalmente mais eficiente para grandes datasets

**Como melhorar o sistema de recomendação?**

1. **Sistema Híbrido**: Combinar ambas abordagens
2. **Content-Based Filtering**: Usar gênero, diretor, atores
3. **Deep Learning**: Autoencoders ou redes neurais
4. **Context Awareness**: Considerar tempo e localização
5. **Feedback Implícito**: Cliques, tempo de visualização

**Resultados**: Ambas abordagens mostraram-se viáveis, com a baseada em itens sendo mais escalável.